This blog post is the second in line of the series of posts discussing on how to create an AI (Artificial intelligent) trading bot.  We try to make the individual blog post to be self content, for this reason we will give you a recap of the problem we are solving.  We are creating smart trading bot using AI, we will call the bot chambot.  The steps invoved are in the  flow diagram below 

![](../images/flow_diagram.png)


* Already discuss how to fetch data from Binance in [here](https://chambox.github.io/1_AI_trading_bot_fetch_data/)
* This blog is about point 2., creating a target variable. 
